In [20]:
import pandas as pd
import json
import os

# Giriş ve çıkış dosya yolları
input_path = "C:/Users/LENOVO/Desktop/patient_assistant/data/raw/clinical_conditions_symptoms/Disease_symptom_and_patient_profile_dataset.csv"
output_path = "processed/clinical_conditions_info.jsonl"

# Gerekirse çıkış klasörünü oluştur
os.makedirs("processed", exist_ok=True)



In [22]:
# Belirti sütunları
symptom_columns = ["Fever", "Cough", "Fatigue", "Difficulty Breathing"]

# CSV dosyasını oku
df = pd.read_csv(input_path)
df.head()


,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive


In [24]:
# 'Yes' olan belirtileri listele
df["Symptoms"] = df.apply(
    lambda row: [col for col in symptom_columns if row[col] == "Yes"], axis=1
)
df[["Disease", "Symptoms"]].head()


,Disease,Symptoms
0,Influenza,"[Fever, Fatigue, Difficulty Breathing]"
1,Common Cold,"[Cough, Fatigue]"
2,Eczema,"[Cough, Fatigue]"
3,Asthma,"[Fever, Cough, Difficulty Breathing]"
4,Asthma,"[Fever, Cough, Difficulty Breathing]"


In [26]:
# Yaş grubunu sınıflandıran fonksiyon
def categorize_age(age):
    if age < 40:
        return "Young"
    elif 40 <= age < 60:
        return "Middle-aged"
    elif 60 <= age < 75:
        return "Senior"
    else:
        return "Elderly"

# Uygula
df["Age_Group"] = df["Age"].apply(categorize_age)
df["Symptoms_Tuple"] = df["Symptoms"].apply(tuple)


In [28]:
# Temiz veri çerçevesi
df_clean = df[[
    "Disease", "Symptoms_Tuple", "Age_Group", "Gender", "Blood Pressure", "Cholesterol Level"
]].drop_duplicates()

# Listeye dönüştür
df_clean["Symptoms"] = df_clean["Symptoms_Tuple"].apply(list)
df_clean = df_clean.drop(columns=["Symptoms_Tuple"])
df_clean.head()


,Disease,Age_Group,Gender,Blood Pressure,Cholesterol Level,Symptoms
0,Influenza,Young,Female,Low,Normal,"[Fever, Fatigue, Difficulty Breathing]"
1,Common Cold,Young,Female,Normal,Normal,"[Cough, Fatigue]"
2,Eczema,Young,Female,Normal,Normal,"[Cough, Fatigue]"
3,Asthma,Young,Male,Normal,Normal,"[Fever, Cough, Difficulty Breathing]"
5,Eczema,Young,Female,Normal,Normal,[Fever]


In [30]:
info_cards = []

for disease in df_clean["Disease"].unique()[:10]:
    data = df_clean[df_clean["Disease"] == disease]
    all_symptoms = sum(data["Symptoms"].tolist(), [])
    common_symptoms = list(set(all_symptoms))[:5]

    card = {
        "disease": disease,
        "symptoms": common_symptoms,
        "common_in": data["Age_Group"].mode()[0],
        "gender_distribution": f"Mostly {data['Gender'].mode()[0]}",
        "risk_factors": [
            data["Blood Pressure"].mode()[0],
            data["Cholesterol Level"].mode()[0]
        ],
        "misconception": f"{disease} genellikle hafif bir durum sanılsa da ciddi sonuçlara yol açabilir.",
        "advice": f"Eğer belirtileriniz {', '.join(common_symptoms)} ise bir doktora danışmanız önerilir."
    }

    info_cards.append(card)

info_cards[:2]  # Örnek olarak ilk 2 bilgi kartını göster


[{'disease': 'Influenza',
  'symptoms': ['Cough', 'Fatigue', 'Fever', 'Difficulty Breathing'],
  'common_in': 'Young',
  'gender_distribution': 'Mostly Female',
  'risk_factors': ['Normal', 'Normal'],
  'misconception': 'Influenza genellikle hafif bir durum sanılsa da ciddi sonuçlara yol açabilir.',
  'advice': 'Eğer belirtileriniz Cough, Fatigue, Fever, Difficulty Breathing ise bir doktora danışmanız önerilir.'},
 {'disease': 'Common Cold',
  'symptoms': ['Cough', 'Fatigue', 'Fever', 'Difficulty Breathing'],
  'common_in': 'Young',
  'gender_distribution': 'Mostly Female',
  'risk_factors': ['High', 'Normal'],
  'misconception': 'Common Cold genellikle hafif bir durum sanılsa da ciddi sonuçlara yol açabilir.',
  'advice': 'Eğer belirtileriniz Cough, Fatigue, Fever, Difficulty Breathing ise bir doktora danışmanız önerilir.'}]

Belirti sütunlarını tanımla ve Symptoms sütununu oluştur: CSV'de "Fever", "Cough", "Fatigue", "Difficulty Breathing" sütunlarını kontrol edip, "Yes" olanları Symptoms listesine çevir.

In [32]:
# JSONL olarak diske yaz
with open(output_path, "w", encoding="utf-8") as f:
    for card in info_cards:
        json.dump(card, f, ensure_ascii=False)
        f.write("\n")

print(f"{len(info_cards)} bilgi kartı başarıyla '{output_path}' dosyasına yazıldı.")


10 bilgi kartı başarıyla 'processed/clinical_conditions_info.jsonl' dosyasına yazıldı.


 Yaş grubunu sınıflandır (Young, Middle-aged, Senior, Elderly)

In [ ]:
symptom_columns = ["Fever", "Cough", "Fatigue", "Difficulty Breathing"]

df["Symptoms"] = df.apply(
    lambda row: [col for col in symptom_columns if row[col] == "Yes"], axis=1
)

df[["Disease", "Symptoms"]].head()


Tekrarlayan satırları temizle ve formatla

In [40]:
df["Symptoms_Tuple"] = df["Symptoms"].apply(tuple)

df_clean = df[[
    "Disease", "Symptoms_Tuple", "Age_Group", "Gender", "Blood Pressure", "Cholesterol Level"
]].drop_duplicates()

df_clean["Symptoms"] = df_clean["Symptoms_Tuple"].apply(list)
df_clean = df_clean.drop(columns=["Symptoms_Tuple"])

df_clean.head()


,Disease,Age_Group,Gender,Blood Pressure,Cholesterol Level,Symptoms
0,Influenza,Young,Female,Low,Normal,"[Fever, Fatigue, Difficulty Breathing]"
1,Common Cold,Young,Female,Normal,Normal,"[Cough, Fatigue]"
2,Eczema,Young,Female,Normal,Normal,"[Cough, Fatigue]"
3,Asthma,Young,Male,Normal,Normal,"[Fever, Cough, Difficulty Breathing]"
5,Eczema,Young,Female,Normal,Normal,[Fever]


Bilgi kartlarını oluştur (ilk 10 hastalık için)

In [43]:
info_cards = []

for disease in df_clean["Disease"].unique()[:10]:
    data = df_clean[df_clean["Disease"] == disease]
    all_symptoms = sum(data["Symptoms"].tolist(), [])
    common_symptoms = list(set(all_symptoms))[:5]

    card = {
        "disease": disease,
        "symptoms": common_symptoms,
        "common_in": data["Age_Group"].mode()[0],
        "gender_distribution": f"Mostly {data['Gender'].mode()[0]}",
        "risk_factors": [
            data["Blood Pressure"].mode()[0],
            data["Cholesterol Level"].mode()[0]
        ],
        "misconception": f"{disease} genellikle hafif bir durum sanılsa da ciddi sonuçlara yol açabilir.",
        "advice": f"Eğer belirtileriniz {', '.join(common_symptoms)} ise bir doktora danışmanız önerilir."
    }

    info_cards.append(card)

info_cards[:2]  # İlk 2 tanesini kontrol edelim


[{'disease': 'Influenza',
  'symptoms': ['Cough', 'Fatigue', 'Fever', 'Difficulty Breathing'],
  'common_in': 'Young',
  'gender_distribution': 'Mostly Female',
  'risk_factors': ['Normal', 'Normal'],
  'misconception': 'Influenza genellikle hafif bir durum sanılsa da ciddi sonuçlara yol açabilir.',
  'advice': 'Eğer belirtileriniz Cough, Fatigue, Fever, Difficulty Breathing ise bir doktora danışmanız önerilir.'},
 {'disease': 'Common Cold',
  'symptoms': ['Cough', 'Fatigue', 'Fever', 'Difficulty Breathing'],
  'common_in': 'Young',
  'gender_distribution': 'Mostly Female',
  'risk_factors': ['High', 'Normal'],
  'misconception': 'Common Cold genellikle hafif bir durum sanılsa da ciddi sonuçlara yol açabilir.',
  'advice': 'Eğer belirtileriniz Cough, Fatigue, Fever, Difficulty Breathing ise bir doktora danışmanız önerilir.'}]

JSONL dosyasına yaz

In [50]:
output_path = "processed/clinical_conditions_info2.jsonl"

with open(output_path, "w", encoding="utf-8") as f:
    for card in info_cards:
        json.dump(card, f, ensure_ascii=False)
        f.write("\n")

print(f"{len(info_cards)} bilgi kartı başarıyla '{output_path}' dosyasına yazıldı.")


10 bilgi kartı başarıyla 'processed/clinical_conditions_info2.jsonl' dosyasına yazıldı.
